In [1]:
pip install torch torchvision torchaudio


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch 
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler



df_tracks = pd.read_csv("high_popularity_spotify_data.csv")
#df_tracks = pd.read_csv("low_popularity_spotify_data.csv", index_col = 0)

#features are energy, tempo, danceability, loudness, liveness, valence, time_signature, speechiness



features = ["energy", "tempo", "danceability", "loudness",
                     "liveness", "valence",  "time_signature",
                    "speechiness"]


X = df_tracks[features]
y = df_tracks["playlist_genre"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)




label_encoder = LabelEncoder()


y_train = torch.tensor(label_encoder.fit_transform(y_train), dtype=torch.long)
y_test  = torch.tensor(label_encoder.transform(y_test), dtype=torch.long)



device = torch.device("cude" if torch.cuda.is_available() else "cpu")
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)



class MLP(nn.Module):
    def __init__(self, input_size=8, hidden1=64, hidden2=32, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden2, num_classes),
        )
    def forward(self, x):
        return self.net(x)

num_classes = len(np.unique(y_train.cpu().numpy()))
model = MLP(input_size=8, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


model.eval()
with torch.no_grad():
    preds = model(X_test)
    predicted = preds.argmax(dim=1)
    acc = (predicted == y_test).float().mean()
    print(f"\nTest Accuracy: {acc.item()*100:.2f}%")

predicted_genres = label_encoder.inverse_transform(predicted.cpu().numpy())





Epoch [5/50], Loss: 3.3355
Epoch [10/50], Loss: 3.2864
Epoch [15/50], Loss: 3.2243
Epoch [20/50], Loss: 3.1470
Epoch [25/50], Loss: 3.0639
Epoch [30/50], Loss: 2.9569
Epoch [35/50], Loss: 2.8399
Epoch [40/50], Loss: 2.7128
Epoch [45/50], Loss: 2.6215
Epoch [50/50], Loss: 2.5402

Test Accuracy: 34.91%
